In [20]:
import openai
import re
import pandas as pd
from datetime import datetime, timedelta
# .env파일에서 OPENAI_API_KEY 가져오기
import os
from dotenv import load_dotenv

In [21]:
load_dotenv()  # 이 코드를 추가하여 .env 파일을 명시적으로 로드합니다.
api_key = os.getenv('OPENAI_API_KEY')
# openai api_key 설정
openai.api_key = api_key

# print("api_key:", api_key)

In [22]:
# 특수 문자 제거 함수
def clean_text(text):
    text = re.sub(r'[^ㄱ-ㅎ가-힣a-zA-Z0-9\s]', '', text)
    return text

In [36]:
# 언어 감지 함수
def detect_language(text):
    response = openai.chat.completions.create(
        model="gpt-3.5-turbo",
        messages=[
            {"role": "system", "content": "You are a language detection assistant."},
            {"role": "user", "content": f"다음 텍스트의 주요 언어를 감지해 주세요: {text}"}
        ],
        max_tokens=5
    )
    print(response)
    return response.choices[0].message.content.strip()

In [24]:
# 텍스트 임베딩 생성 함수
def get_chatgpt_embedding(text):
    response = openai.Embedding.create(
        model="text-embedding-ada-002",
        input=text
    )
    return response['data'][0]['embedding']

In [25]:
# ISO 8601 형식의 duration을 초 단위로 변환하는 함수
def parse_duration(duration):
    duration = duration.replace('PT', '')
    hours = minutes = seconds = 0
    if 'H' in duration:
        hours, duration = duration.split('H')
        hours = int(hours)
    if 'M' in duration:
        minutes, duration = duration.split('M')
        minutes = int(minutes)
    if 'S' in duration:
        seconds = int(duration.rstrip('S'))
    total_seconds = hours * 3600 + minutes * 60 + seconds
    return total_seconds

In [26]:
# 데이터 전처리를 위한 데이터 적재
data = pd.read_csv('./youtube_channel_comments_data_20240602_033255.csv')


In [27]:
# 데이터 분석을 위해 날짜 데이터 열(channelPublishedAt, videoPublishedAt, commentPublishedAt)을 
# 오브젝트타입에서  datetime으로 변경
data['channelPublishedAt'] = pd.to_datetime(data['channelPublishedAt'], format='mixed')
data['videoPublishedAt'] = pd.to_datetime(data['videoPublishedAt'], format='mixed')
data['commentPublishedAt'] = pd.to_datetime(data['commentPublishedAt'], format='mixed')

In [28]:
# 비디오(영상)별 재생시간을 나타내는 duration 컬럼에 대한 전처리
# duration 컬럼의 데이터를 초 단위로 변환
data['duration'] = data['duration'].apply(parse_duration)


In [29]:
# 누락된 데이터 처리
data.fillna({'commentText': 'No comment'}, inplace=True)

In [30]:
# 데이터 타입 검증 및 변환
data['subscriberCount'] = data['subscriberCount'].astype(int)
data['viewCount'] = data['viewCount'].astype(int)
data['likeCount'] = data['likeCount'].astype(int)
data['dislikeCount'] = data['dislikeCount'].astype(int)
data['commentLikeCount'] = data['commentLikeCount'].astype(int)

In [37]:
# 댓글, 답글 데이터 전처리
data['cleaned_commentText'] = data['commentText'].apply(clean_text)
data['language'] = data['cleaned_commentText'].apply(detect_language)

# 텍스트 임베딩 생성
data['embedding'] = data['cleaned_commentText'].apply(get_chatgpt_embedding)

ChatCompletion(id='chatcmpl-9VagQptDkcHBMEHUUY6gzw6gxj6ey', choices=[Choice(finish_reason='length', index=0, logprobs=None, message=ChatCompletionMessage(content='입력하신', role='assistant', function_call=None, tool_calls=None))], created=1717316206, model='gpt-3.5-turbo-0125', object='chat.completion', system_fingerprint=None, usage=CompletionUsage(completion_tokens=4, prompt_tokens=74, total_tokens=78))
ChatCompletion(id='chatcmpl-9VagRhl2rTY1QlJbnpbzS2mFyycrF', choices=[Choice(finish_reason='length', index=0, logprobs=None, message=ChatCompletionMessage(content='입력하신', role='assistant', function_call=None, tool_calls=None))], created=1717316207, model='gpt-3.5-turbo-0125', object='chat.completion', system_fingerprint=None, usage=CompletionUsage(completion_tokens=4, prompt_tokens=61, total_tokens=65))
ChatCompletion(id='chatcmpl-9VagRn6FSuuNtCoKXO8M3T9m3O6Go', choices=[Choice(finish_reason='length', index=0, logprobs=None, message=ChatCompletionMessage(content='입력된 텍', role='assistant',

KeyboardInterrupt: 

In [ ]:
# 영상 제목 데이터 전처리
data['videoTitle'] = data['videoTitle'].apply(clean_text)


In [ ]:
# 중복 데이터 제거
data.drop_duplicates(inplace=True)

In [ ]:
# 카테고리 데이터 처리
data['channelId'] = data['channelId'].astype('category')
data['videoId'] = data['videoId'].astype('category')
data['commentId'] = data['commentId'].astype('category')

In [ ]:
# 결과 저장
data.to_csv('./youtube_channel_comments_cleaneddata.csv', index=False)

# 데이터 확인
print(data.head())